<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Mistral_in_AWS_with_TWOMODELS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PREPARATION AND INTRODUCTION TBD##

In [1]:

# https://aws.amazon.com/blogs/machine-learning/mistral-7b-foundation-models-from-mistral-ai-are-now-available-in-amazon-sagemaker-jumpstart/
# https://github.com/mistralai/mistral-src

#It looks great
# https://mistral.ai/news/mixtral-of-experts/

# https://www.nytimes.com/2023/12/10/technology/mistral-ai-funding.html

# https://aws.amazon.com/blogs/machine-learning/mistral-7b-foundation-models-from-mistral-ai-are-now-available-in-amazon-sagemaker-jumpstart/

!pip install sagemaker
!pip install boto3
!pip install --upgrade urllib3
!pip install colab-env --upgrade

import colab_env
import boto3
import sagemaker
from sagemaker.jumpstart.model import JumpStartModel
import os

aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
region=os.getenv("AWS_DEFAULT_REGION")
output=os.getenv("AWS_DEFAULT_OUTPUT")

iam_client = boto3.client("iam")

role = iam_client.get_role(
    RoleName=os.getenv("ROLENAME")
)

ROLE_ARN = role['Role']['Arn']

print('huggingface-llm-mistral-7b-instruct - JumpStartModel')
model_version='2.0.0'
model = JumpStartModel(model_id="huggingface-llm-mistral-7b-instruct", model_version='2.0.0', role=ROLE_ARN)
predictor = model.deploy()

print()
print()

print('meta-textgeneration-llama-2-7b-f - JumpStartModel')
model_id, model_version = "meta-textgeneration-llama-2-7b-f", "2.0.4"

(model_id, model_version,) = (
    "meta-textgeneration-llama-2-7b",
    "2.1.8",
)

model_llama = JumpStartModel(model_id=model_id, model_version=model_version, role=ROLE_ARN)
predictor_llama = model_llama.deploy()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 877.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

For forward compatibility, pin to model_version='2.*' in your JumpStartModel or JumpStartEstimator definitions. Note that major version upgrades may have different EULA acceptance terms and input/output signatures.


---------------!

In [ ]:
# https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_amazon_algorithms/jumpstart-foundation-models/llama-2-text-completion.ipynb

def print_response(payload, response):
    print(payload["inputs"])
    print(f"> {response[0]['generation']}")
    print("\n==================================\n")

#"inputs": "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.",

%time
payload = {
    "inputs": "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.",
    #"inputs": "I believe the meaning of life is",
    "parameters": {
        "max_new_tokens": 1024,
        "top_p": 0.9,
        "temperature": 0.6,
        "return_full_text": False,
    },
}

try:
    response = predictor_llama.predict(payload, custom_attributes='accept_eula=true')
    print_response(payload, response)
except Exception as e:
    print(e)


##PROMPT AND VALIDATION""

In [13]:

def predict_sentiment(prompt,predictor,modelid):
    if modelid == 0:
       INPUT= "<s>[INST]  " + prompt + " [/INST]"
    else:
       INPUT= "%s"%prompt

    payload = {
        #"inputs": "<s>[INST]  " + prompt + " [/INST]",
        "inputs": "%s"%INPUT,
        "parameters": {
            "do_sample": True,
            "top_p": 0.9,
            "temperature": 0.9,
            "max_new_tokens": 512,
            "return_full_text": False,
            #"stop": ["<|endoftext|>", "</s>"]
        },
    }
    return predictor.predict(payload,custom_attributes="accept_eula=true")

prompt0 = "As a data scientist, can you explain the concept of regularization in machine learning?"
#prompt2 = "I am currently have two bananas. i ate one yestesday. how many do i have now?"
print()
print()
print('TEST0: Predict the following words in the sequence')
#prompt2 = "This new music video was incredibile"
#prompt2 = "I currently have two bananas and one apple in my basket. i ate the apple now. how many items are in my basket?"
print()
print('Prompt #2: %s'%prompt0)
print()
sentiment = predict_sentiment(prompt0,predictor,0)
#print(sentiment)
print('OUTPUT PROMPT0:')
print()
print(f"Answer: {sentiment[0]['generated_text']}")
print()



TEST0: Predict the following words in the sequence

Prompt #2: As a data scientist, can you explain the concept of regularization in machine learning?

OUTPUT PROMPT0:

Answer:  Certainly! Regularization is a technique used in machine learning to prevent overfitting of models. It involves adding a penalty term to the loss function, which discourages the model from fitting the training data too closely and instead encourages it to have a more general solution that can generalize to new, unseen data.

Overfitting occurs when a model is too complex and fits the training data too closely, resulting in poor performance on new data. Regularization works by adding a penalty term to the loss function, which encourages the model to have a simpler solution that is less likely to overfit.

There are several types of regularization, including L1 (Lasso) regularization, L2 (Ridge) regularization, and Elastic Net regularization. Each type of regularization works by adding a different penalty term 

In [15]:
#payload = {"inputs": "<s>[INST] Hello! [/INST]"}
#print(predictor.predict(payload))

#prompt = "Tell me about Amazon SageMaker."

#endpoint_name=model.endpoint_name

import json

def predict_sentiment(prompt,predictor,modelid):
    if modelid == 0:
       INPUT= "<s>[INST]  " + prompt + " [/INST]"
    else:
       INPUT= "%s"%prompt

    payload = {
        #"inputs": "<s>[INST]  " + prompt + " [/INST]",
        "inputs": "%s"%INPUT,
        "parameters": {
            "do_sample": True,
            "top_p": 0.9,
            "temperature": 0.9,
            "max_new_tokens": 512,
            "return_full_text": False,
            #"stop": ["<|endoftext|>", "</s>"]
        },
    }
    return predictor.predict(payload,custom_attributes="accept_eula=true")


print('TESTING of the Mistral LLM running hosted in AWS WITH THE CODE RUNNING FROM GOOGLE CLOUD with Google Colab')

print()
print()
print()
print('TEST1 MISTRAL: Mathematics and reasoning')
prompt1='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
print()
print('Prompt #1: %s'%prompt1)
print()
sentiment = predict_sentiment(prompt1,predictor,0)
#s=sentiment[0]
print('OUTPUT PROMPT1:')
print(f"Answer: {sentiment[0]['generated_text']}")
#print(s['generated_text'])
#print(sentiment)
print()



print()
print()
print()
print('TEST1 - LLAMA2: Mathematics and reasoning')
prompt1='I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.'
print()
print('Prompt #1: %s'%prompt1)
print()
sentiment = predict_sentiment(prompt1,predictor_llama,1)
#s=sentiment[0]
print('OUTPUT PROMPT1 - LLAMA2:')
#print(s)
print(f"Answer: {sentiment[0]['generation']}")
#print(s['generation'])
#print(sentiment)
print()

#print(predictor.predict(payload))

#> Input
#Tweet: "I get sad when my phone battery dies."
#Sentiment: Negative

payload = {"inputs": "<s>[INST] Tweet: I get sad when my phone battery dies. [/INST]"}
#print(predictor.predict(payload))

###
#Tweet: "My day has been :+1:"
#Sentiment: Positive

###
#Tweet: "This is the link to the article"
#Sentiment: Neutral

###
#Tweet: "This new music video was incredibile"
#Sentiment:

payload = {"inputs": "<s>[INST] Tweet: My day has been :+1: [/INST]"}
#print(predictor.predict(payload))

payload = {"inputs": "<s>[INST] This is the link to the article [/INST]"}
#print(predictor.predict(payload))

prompt0= "I am currently have two bananas. i ate one yestesday. how many do i have now?"
print()
print()
print('TEST2 MISTRAL: Predict the following words in the sequence')
prompt2 = "This new music video was incredibile"
#prompt2 = "I currently have two bananas and one apple in my basket. how many items are in my basket?"
print()
print('Prompt #2: %s'%prompt2)
print()
sentiment = predict_sentiment(prompt2,predictor,0)
#s=sentiment[0]
print('OUTPUT PROMPT2:')
print(f"Answer: {sentiment[0]['generated_text']}")
#print(s['generated_text'])
#print(sentiment)
print()

prompt3='A rat is a rodent, the most common mammal in the world. Rattus norvegicus is one of the approximately four hundred different kinds of rodents, and it is known by many names, each of which describes a trait or a perceived trait or sometimes a habitat: the earth rat, the roving rat, the barn rat, the fi eld rat, the migratory rat, the house rat, the sewer rat, the water rat, the wharf rat, the alley rat, the gray rat, the brown rat, and the common rat. The average brown rat is large and stocky; it grows to be approximately sixteen inches \
long from its nose to its tail—the size of a large adult human male’s foot—and weighs about a pound, though brown rats have been measured by scientists and exterminators at twenty inches and up to two pounds. The brown rat is sometimes confused with the black rat, or Rattus rattus, which is smaller and once inhabited New York City and all of the cities of America but, since Rattus norvegicus pushed it out, is now relegated to a minor role. (The two species still survive alongside each other in some Southern coastal cities and on the West Coast, in places like Los Angeles, for example, where the black rat lives in attics and palm trees.) The black rat is always a very dark gray, almost black, \
and the brown rat is gray or brown, with a belly that can be light gray, yellow, or even a pure-seeming white. One spring, beneath the Brooklyn Bridge, I saw a red-haired brown rat that had been run over by a car. Both pet rats and laboratory rats are Rattus norvegicus, but they are not wild and therefore, I would emphasize, not the subject of this book. Sometimes pet rats are called fancy rats. But if anyone has picked up this book to learn about fancy rats, then they should put this book down right away; none of the rats mentioned herein are at all fancy. \
Rats are nocturnal, and out in the night the brown rat’s eyes are small and black and shiny; when a fl ashlight shines into them in the dark, the eyes of a rat light up like the eyes of a deer. Though it forages* in darkness, the brown rat has poor eyesight. It makes up for this with, fi rst of all, an excellent sense of smell. . . . They have an excellent sense of taste, detecting the most minute amounts of poison, down to one part per million. A brown rat has strong feet, the two front paws each equipped with four clawlike nails, the rear paws even longer and stronger. It can run and climb with squirrel-like agility. It is an excellent swimmer, surviving in rivers and bays, in sewer streams and toilet bowls. \
The brown rat’s teeth are yellow, the front two incisors being especially long and sharp, like buckteeth. When the brown rat bites, its front two teeth spread apart. When it gnaws, a fl ap of skin plugs the space behind its incisors. Hence, when the rat gnaws on indigestible materials—concrete or steel, for example—the shavings don’t go down the rat’s throat and kill it. Its incisors grow at a rate of fi ve inches per year. Rats always gnaw, and no one is certain why—there are few modern rat studies. It is sometimes erroneously stated that the rat gnaws solely to limit the length of its incisors, which would otherwise grow out of its head, but this is not the case: the incisors wear down naturally. In terms of hardness, the brown rat’s teeth are stronger than aluminum, copper, lead, and iron. They are comparable to steel. With the alligator-like structure of their jaws, rats can exert a biting pressure of up to seven thousand pounds per square inch. Rats, like mice, seem to be attracted to wires—to utility wires, computer wires, wires in vehicles, in addition to gas and water pipes. One rat expert theorizes that wires may be attractive to rats because of their resemblance to vines and the stalks of plants; cables are the vines of the city. By one estimate, 26 percent of all electric-cable breaks and 18 percent of all phone-cable disruptions are caused by rats. According to one study, as many as 25 percent of all fi res of unknown origin are rat-caused. Rats chew electrical cables. Sitting in a nest of tattered rags and newspapers, in the fl oorboards of an old tenement, \
a rat gnaws the head of a match—the lightning in the city forest.When it is not gnawing or feeding on trash, the brown rat digs. Anywhere there is dirt in a city, brown rats are likely to be digging—in parks, in fl owerbeds, in little dirt-poor backyards. They dig holes to enter buildings and to make nests. Rat nests can be in the floorboards of apartments, in the waste-stuffed corners of subway stations, in sewers, or beneath old furniture in basements. “Cluttered and unkempt alleyways in cities provide ideal rat habitat, especially those alleyways associated with food-serving establishments,” writes Robert Corrigan in Rodent Control, a pest control manual. “Alley rats can forage safely within the shadows created by the alleyway, as well as quickly retreat to the safety of cover in these narrow channels.” Often, rats burrow under concrete sidewalk slabs. Entrance to a typical under-the-sidewalk rat’s nest is gained through a two-inch-wide hole—their skeletons collapse and they can squeeze into a hole as small as three quarters of an inch wide, the average width of their skull. This tunnel then travels about a foot down to where it widens into a nest or den. The den is lined with soft debris, often shredded plastic garbage or shopping bags, but sometimes even grasses or plants; some rat nests have been found stuffed with the gnawed shavings of the wood-based, spring-loaded snap traps that are used in attempts to kill them. The back of the den then narrows into a long tunnel that opens up on another hole back on the street. This second hole is called a bolt hole; it is an emergency exit. A bolt hole is typically covered lightly with dirt or trash—camoufl age. Sometimes there are networks of burrows, which can stretch beneath a few concrete squares on a sidewalk, or a number of backyards, or even an entire city block—when Rattus norvegicus fi rst came to Selkirk, England, in 1776, there were so many burrows that people feared the town might sink. Rats can also nest in basements, sewers, manholes, abandoned pipes of any kind, fl oorboards, or any hole or depression. “Often,” Robert Corrigan writes, “‘city rats’ will live unbeknownst to people right beneath their feet.” \
Rats also inhabit subways, as most people in New York City and any city with a subway system are well aware. Every once in a while, there are reports of rats boarding trains, \
but for the most part rats stay on the tracks—subway workers \
I have talked to refer to rats as “track rabbits.” People tend to think that the subways are fi lled with rats, \
but in fact rats are not everywhere in the system; they live in the subways according to the supply of discarded \
human food and sewer leaks. Sometimes, rats use the subway purely for nesting purposes; they fi nd ways through \
the walls of the subway stations leading from the tracks to the restaurants and stores on the street—the vibrations of \
 subway trains tend to create rat-size cracks and holes. Many subway rats tend to live near stations that are themselves near fast-food restaurants. At the various subway stations near Herald Square, for example, people come down from the streets and throw the food that they have not eaten onto the tracks, along with newspapers and soda bottles and, I have noticed, thousands of nolonger-charged AA batteries, waiting to leak acid. The rats eat freely from the waste and sit at the side of the little streams of creamy brown sewery water that fl ows between the rails. They sip the water the way rats do, either with their front paws or by scooping it up with their incisors. \
What is the most likely reason the author states, "cables are the vines of the city"?'

print()
print('TEST3 MISTRAL: MCAS Grade 10 English Language Arts Reading Comprehension test')
print()
print('Prompt #3: %s'%prompt3)
print()
sentiment = predict_sentiment(prompt3,predictor,0)
#s=sentiment[0]
print('OUTPUT PROMPT3:')
print(f"Answer: {sentiment[0]['generated_text']}")
#print(s['generated_text'])
#print(sentiment)
print()

prompt4='Which country has the most natural lakes? Answer with only the country name.'
#prompt4='who was the last United States president?'
#prompt4='What is the capital of the Unites States?'
#prompt4='Who was the first president of the United States?'
#prompt4='who is obama'
#prompt4='write an essay about AI'
print()
print('TEST4 MISTRAL: Knowledge retrieval')
print()
print('Prompt #4: %s'%prompt4)
print()
sentiment = predict_sentiment(prompt4,predictor,0)
#s=sentiment[0]
print('OUTPUT PROMPT4:')
print(f"Answer: {sentiment[0]['generated_text']}")
#print(s['generated_text'])
#print(sentiment[0])

print()
print()
print('TEST4 - LLAMA2: Knowledge retrieval')
print()
print('Prompt #4: %s'%prompt4)
print()
sentiment = predict_sentiment(prompt4,predictor_llama,1)
print('OUTPUT PROMPT4 - LLAMA2:')
print(f"Answer: {sentiment[0]['generation']}")
print()


TESTING of the Mistral LLM running hosted in AWS WITH THE CODE RUNNING FROM GOOGLE CLOUD with Google Colab



TEST1 MISTRAL: Mathematics and reasoning

Prompt #1: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.

OUTPUT PROMPT1:
Answer:  Let's break down the problem:

1. You bought six ice cream cones, with each cone costing $1.25.
2. That means the total cost of the ice cream was:
   Total cost = Number of cones * Cost per cone
   = 6 * $1.25
   = $7.50
3. You paid for the ice cream with a $10 bill, which is $2.50 more than the total cost of the ice cream ($10 - $7.50 = $2.50).
4. Therefore, you got $2.50 back.

The answer is that you got $2.50 back.




TEST1 - LLAMA2: Mathematics and reasoning

Prompt #1: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.

OUTPUT PROMPT1 - LLAMA2:
Answer: 
A.

##CLEAN UP##

In [16]:

#  Frank Morales created this cell on December 14, 2023; it fully allows automatically the deletion of endpoints, models, and endpoint configurations.
## TODO create a function for the loop

aws_region = 'us-east-1'
import boto3

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

print('')
print('Endpoints')
response=sagemaker_client.list_endpoints()
number_of_endpoints=len(response['Endpoints'])
for i in range(number_of_endpoints):
    print(response['Endpoints'][i]['EndpointName'])
    endpoint_name=response['Endpoints'][i]['EndpointName']
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)


print('')
print('Models')
response_models=sagemaker_client.list_models()
for i in range(len((response_models['Models']))):
    print(response_models['Models'][i]['ModelName'])
    sagemaker_client.delete_model(ModelName=response_models['Models'][i]['ModelName'])


print('')
print('EndpointConfigs')
response_configs=sagemaker_client.list_endpoint_configs()
for i in range(len((response_configs['EndpointConfigs']))):
    print(response_configs['EndpointConfigs'][i]['EndpointConfigName'])
    sagemaker_client.delete_endpoint_config(EndpointConfigName=response_configs['EndpointConfigs'][i]['EndpointConfigName'])


Endpoints
meta-textgeneration-llama-2-7b-2023-12-17-23-45-51-522
hf-llm-mistral-7b-instruct-2023-12-17-23-40-47-946

Models
meta-textgeneration-llama-2-7b-2023-12-17-23-45-51-518
hf-llm-mistral-7b-instruct-2023-12-17-23-40-47-944

EndpointConfigs
meta-textgeneration-llama-2-7b-2023-12-17-23-45-51-522
hf-llm-mistral-7b-instruct-2023-12-17-23-40-47-946
